In [2]:
!git clone https://github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 43.69 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Checking out files: 100% (13996/13996), done.


In [3]:
import csv
data={}
with open('/content/facial_expressions/data/legend.csv') as f:
  reader=csv.reader(f)
  next(reader)
  for row in reader:
    key=row[2].lower()
    if key in data:
      data[key].append(row[1])
    else:
      data[key]=[row[1]]

In [4]:
#Directory structure -- imagedatagenerator class from keras 
emotion_list=list(data.keys())
emotion_list
#Smaller and upper case. Don't want like that
#8 sub-directories

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [5]:
import os
os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

In [6]:
for emotion in emotion_list:
  os.mkdir(os.path.join('master_data/training',emotion))
  os.mkdir(os.path.join('master_data/testing',emotion))

In [7]:
from shutil import copyfile
split_size=0.8

for emotion,images in data.items():
  train_size=int(split_size*len(images))
  train_images=images[:train_size]
  test_images=images[train_size:]
  for image in train_images:
    source=os.path.join('/content/facial_expressions/images',image)
    dest=os.path.join('/content/master_data/training',emotion,image)
    copyfile(source,dest)

  for image in test_images:
    source=os.path.join('/content/facial_expressions/images',image)
    dest=os.path.join('/content/master_data/testing',emotion,image)
    copyfile(source,dest)



In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [9]:
model=tf.keras.models.Sequential([
  Conv2D(16,(3,3),activation='relu',input_shape=(100,100, 3)),
  MaxPooling2D(2,2),
  Conv2D(32,(3,3),activation='relu'),
  MaxPooling2D(2,2),
  Conv2D(64,(3,3),activation='relu'),
  MaxPooling2D(2,2),   
  Flatten(),
  Dense(512,activation='relu'),
  Dense(8,activation='softmax')                             
])
model.compile(optimizer=Adam(lr=0.01),loss='categorical_crossentropy',metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0

In [10]:
train_dir='/content/master_data/training'
test_dir='/content/master_data/testing'

train_datagen=ImageDataGenerator(rescale=1.0/255)
train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(100,100),
    class_mode='categorical',
    batch_size=128    
)

test_datagen=ImageDataGenerator(rescale=1.0/255)
test_generator=train_datagen.flow_from_directory(
    test_dir,
    target_size=(100,100),
    class_mode='categorical',
    batch_size=128    
)


Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [11]:
es=EarlyStopping(monitor='val_acc',patience=2,min_delta=0.01)

In [12]:
model.fit_generator(train_generator,
                    epochs=10,
                    verbose=1,
                    validation_data=test_generator,
                    callbacks=[es])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
86/86 [==============================] - 15s 172ms/step - loss: 1.4874 - acc: 0.4908 - val_loss: 1.0428 - val_acc: 0.5011
Epoch 2/10
86/86 [==============================] - 14s 165ms/step - loss: 1.0387 - acc: 0.4995 - val_loss: 1.0425 - val_acc: 0.5011
Epoch 3/10
86/86 [==============================] - 14s 165ms/step - loss: 1.0401 - acc: 0.4980 - val_loss: 1.0437 - val_acc: 0.5011


In [13]:
# Trying to tweak some hyperparameters
model=tf.keras.models.Sequential([
  Conv2D(16,(3,3),activation='relu',input_shape=(100,100, 3)),
  MaxPooling2D(2,2),
  Conv2D(32,(3,3),activation='relu'),
  MaxPooling2D(2,2),
  Conv2D(64,(3,3),activation='relu'),
  MaxPooling2D(2,2),   
  Flatten(),
  Dense(1024,activation='relu'),
  Dense(8,activation='softmax')                             
])
model.compile(optimizer=Adam(lr=0.01),loss='categorical_crossentropy',metrics=['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)             

In [14]:
model.fit_generator(train_generator,
                    epochs=10,
                    verbose=1,
                    validation_data=test_generator,
                    callbacks=[es])

Epoch 1/10
86/86 [==============================] - 15s 169ms/step - loss: 1.5962 - acc: 0.4942 - val_loss: 1.0220 - val_acc: 0.5011
Epoch 2/10
86/86 [==============================] - 14s 167ms/step - loss: 0.9613 - acc: 0.6076 - val_loss: 0.9403 - val_acc: 0.6379
Epoch 3/10
86/86 [==============================] - 14s 163ms/step - loss: 0.8040 - acc: 0.7184 - val_loss: 0.9888 - val_acc: 0.6222
Epoch 4/10
86/86 [==============================] - 14s 165ms/step - loss: 0.7589 - acc: 0.7409 - val_loss: 1.0420 - val_acc: 0.6182
